In [1]:
import re
with open('input19.txt') as csvfile:
    blueprints_str = [[re.findall('Each (obsidian|geode|ore|clay) robot costs', item) + re.findall('(\d+)\s(obsidian|geode|ore|clay)', item) for item in line.strip()[:-1].split('.')] for line in csvfile.readlines()]

In [2]:
blueprints = []
for blueprint in blueprints_str:
    robot_factory = {}
    for robot_construction in blueprint:
        robot_factory[robot_construction[0]] = {y: int(x) for x, y in robot_construction[1:]}
    blueprints.append(robot_factory)

In [3]:
def comparable_value(ores_per_robot, minutes_left, element):
    return sum(element[0][material]*ores_per_robot[material]*minutes_left + element[1][material]*ores_per_robot[material] for material in ['ore','clay','obsidian','geode'])

def ores_per_material(blueprint, material):
    answer = 0
    costs = blueprint[material]
    for cost in costs:
        if cost == 'ore':
            answer += blueprint[material][cost]
        else:
            answer += ores_per_material(blueprint, cost)*blueprint[material][cost]
    return answer

In [4]:
def blueprint_epoch(blueprint, minutes=24):
    materials = ['ore','clay','obsidian','geode']
    ores_per_robot = {material: ores_per_material(blueprint, material) for material in materials}

    answer = []
    stack = [({material: 1 if material == 'ore' else 0 for material in materials}, {material: 1 if material == 'ore' else 0 for material in materials})]
    for i in range(minutes-1):
        minutes_left = minutes-i-1
        new_stack = []
        while stack:
            current_constructed_robots, current_collected_materials = stack.pop()
            for robot_construction in blueprint:
                enough_material_to_build = all([blueprint[robot_construction][needed_material] <= current_collected_materials[needed_material] for needed_material in blueprint[robot_construction]])
                if enough_material_to_build:
                    next_constructed_robots = {kind_of_material: \
                                               current_constructed_robots[kind_of_material] + 1\
                                                   if robot_construction == kind_of_material else \
                                               current_constructed_robots[kind_of_material] \
                                               for kind_of_material in current_constructed_robots}
                    next_collected_materials = {kind_of_material:\
                                                current_collected_materials[kind_of_material] + current_constructed_robots[kind_of_material] - blueprint[robot_construction][kind_of_material]\
                                                    if kind_of_material in blueprint[robot_construction] else \
                                                current_collected_materials[kind_of_material] + current_constructed_robots[kind_of_material]\
                                                for kind_of_material in current_collected_materials}
                    new_stack.append((next_constructed_robots, next_collected_materials))
            for kind_of_material in current_collected_materials:
                current_collected_materials[kind_of_material] += current_constructed_robots[kind_of_material]
            new_stack.append((current_constructed_robots, current_collected_materials))
        
        stack = sorted(new_stack, key=lambda x: comparable_value(ores_per_robot, minutes_left, x), reverse=True)[:5**5] #save best options from last 5 steps
    return max(element[1]['geode'] for element in stack)

Part 01

In [5]:
quality_level = 0
for i, blueprint in enumerate(blueprints):
    quality_level += (i+1) * blueprint_epoch(blueprints[i])
quality_level

1528

Part 02

In [6]:
blueprint_epoch(blueprints[0], minutes=32) * blueprint_epoch(blueprints[1], minutes=32) * blueprint_epoch(blueprints[2], minutes=32)

16926